# 아파트 관리사무소 음성만들기
 - 아파트 관리사무소에서 나오는 기계적 음성은 어떻게 만들까?
 - Tex To Speech (TTS)를 이용해서 구현해보자
 - 참조코드: github.com/kairess/animalese-voice-generator/blob/master/main.ipynb 

In [1]:
ttt = "관리사무소에서 알려드립니다 차량번호 3833차주분 께서는 지금 관리동 305호 로와주시기 바랍니다"

## 1. 패키지 설치

In [2]:
!pip install gtts 
!pip install pydub

In [3]:
from gtts import gTTS
from pydub import AudioSegment
import random, os

## 2. 파일을 저장할 디렉토리 만들기
 - samples: 각 단어별 mp3파일 저장소
 - result: 음성파일 전체에 대한 mp3파일

In [4]:
os.makedirs('samples', exist_ok=True)
os.makedirs('result', exist_ok=True)

In [7]:
letter = "꺄"
lang="ko"
tts = gTTS(letter, lang=lang)
tts.save('samples/%s.mp3' % letter)

In [8]:
letter_sound = AudioSegment.from_mp3('samples/%s.mp3' % letter)
print(letter_sound.raw_data)
letter_sound


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [9]:
string = '안녕하세요 관리사무소에서알려드립니다 309호 차 빼주세요'
lang = 'ko' 
random_factor = 0.35
normal_frame_rate = 25100

In [10]:
result_sound = None
new_sound = letter_sound._spawn(b'\x00' * (normal_frame_rate // 2), overrides={'frame_rate': normal_frame_rate})
result_sound = new_sound if result_sound is None else result_sound + new_sound
result_sound


In [13]:
result_sound = None

for i, letter in enumerate(string):
    if letter == ' ': # fill empty sound
        new_sound = letter_sound._spawn(b'\x00' * (normal_frame_rate // 2), overrides={'frame_rate': normal_frame_rate})
    else: # fill tts sound
        # create tts if sample sound not exist
        if not os.path.isfile('samples/%s.mp3' % letter):
            tts = gTTS(letter, lang=lang)
            tts.save('samples/%s.mp3' % letter)

        # load sample sound
        letter_sound = AudioSegment.from_mp3('samples/%s.mp3' % letter)

        # slice out head and tail part of audio
        raw = letter_sound.raw_data[8000:-8000]

        # adjust pitch around +2
        octaves = 2.0 + random.random() * random_factor
        frame_rate = int(letter_sound.frame_rate * (2.0 ** octaves))
        print('%s - octaves: %.2f, fr: %.d' % (letter, octaves, frame_rate))

        new_sound = letter_sound._spawn(raw, overrides={'frame_rate': frame_rate})

    new_sound = new_sound.set_frame_rate(normal_frame_rate)
    result_sound = new_sound if result_sound is None else result_sound + new_sound
    
result_sound

안 - octaves: 2.03, fr: 97735
녕 - octaves: 2.19, fr: 109278
하 - octaves: 2.22, fr: 112021
세 - octaves: 2.28, fr: 116687
요 - octaves: 2.14, fr: 105714
관 - octaves: 2.16, fr: 107400
리 - octaves: 2.19, fr: 109476
사 - octaves: 2.11, fr: 103725
무 - octaves: 2.09, fr: 102003
소 - octaves: 2.29, fr: 117327
에 - octaves: 2.31, fr: 119051
서 - octaves: 2.03, fr: 97823
알 - octaves: 2.26, fr: 115071
려 - octaves: 2.01, fr: 96464
드 - octaves: 2.09, fr: 102363
립 - octaves: 2.15, fr: 106819
니 - octaves: 2.10, fr: 102860
다 - octaves: 2.23, fr: 112363


In [ ]:
result_sound.export('result/%s.mp3' % string, format='mp3')

<_io.BufferedRandom name='result/안녕하세요 너굴 상회입니다.mp3'>